In [17]:
import tensorflow as tf
import numpy as np
import os
import cv2
import csv as csv
import random, glob
import keras
from sklearn.decomposition import PCA
from sklearn.metrics import confusion_matrix, f1_score
from sklearn.preprocessing import StandardScaler
from keras.models import Sequential
from keras import regularizers
from keras.optimizers import SGD, Adam
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.activations import relu
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA
from keras.layers import Convolution2D, MaxPooling2D, BatchNormalization
from keras.utils import np_utils
from keras.callbacks import EarlyStopping, ModelCheckpoint,ReduceLROnPlateau
from sklearn.utils import shuffle
from keras.datasets import mnist
from matplotlib import pyplot as plt
from keras.layers.advanced_activations import LeakyReLU
from keras.preprocessing.image import ImageDataGenerator
from keras import backend as K
from keras.engine.topology import Layer, InputSpec
from sklearn import svm
from matplotlib import pyplot as plt
from keras.applications.vgg16 import VGG16
from keras.datasets import mnist
from keras import backend as K

In [2]:
class LocalResponseNormalisation(Layer):
    
    def __init__(self, n=5, alpha=0.0005, beta=0.75, k=2, **kwargs):
        self.n =n
        self.alpha = alpha
        self.beta = beta
        self.k = k
        super(LocalResponseNormalisation, self).__init__(**kwargs)
    
    def build(self,input_shape):
        self.shape = input_shape
        super(LocalResponseNormalisation, self).build(input_shape)

    def call(self,x,mask=None):
        _,r,c,f = self.shape
    #   _,f,r,c = self.shape
        squared = K.square(x)
        #print r,c,f,K.image_data_format
        #print squared
        pooled = K.pool2d(squared, (self.n, self.n), strides=(1, 1),padding="same", pool_mode="avg")
        #print pooled       
        summed = K.sum(pooled, axis=1, keepdims=True)
        averaged = self.alpha * K.repeat_elements(summed, r, axis=1)
        denom = K.pow(self.k + averaged, self.beta)
        #print summed,averaged,x,denom
        return x / denom    

    def compute_output_shape(self, input_shape):
        return input_shape

In [13]:
batch_size = 64
num_classes = 10
# epochs = 12
img_rows, img_cols = 28, 28

In [10]:
(x_train, y_train), (x_test, y_test) = mnist.load_data()

11493376/11490434 [==============================] - 5s 0us/step


In [14]:
if K.image_data_format() == 'channels_first':
    x_train = x_train.reshape(x_train.shape[0], 1, img_rows, img_cols)
    x_test = x_test.reshape(x_test.shape[0], 1, img_rows, img_cols)
    input_shape = (1, img_rows, img_cols)
else:
    x_train = x_train.reshape(x_train.shape[0], img_rows, img_cols, 1)
    x_test = x_test.reshape(x_test.shape[0], img_rows, img_cols, 1)
    input_shape = (img_rows, img_cols, 1)

In [15]:
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255
print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')


x_train shape: (60000, 28, 28, 1)
60000 train samples
10000 test samples


In [18]:
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

In [5]:
model = Sequential()
model.add(Convolution2D(8, 5, 5,init = 'he_uniform',input_shape=(28, 28, 1)))
model.add(Activation('relu'))
model.add(Convolution2D(8, 5, 5,init = 'he_uniform'))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2), strides=(2,2)))
model.add(Flatten())
model.add(Dense(128))
model.add(Dense(10))
model.add(Activation('softmax'))

/home/aniket.joshi/.local/lib/python3.5/site-packages/ipykernel_launcher.py:2: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(8, (5, 5), input_shape=(28, 28, 1..., kernel_initializer="he_uniform")`
  
/home/aniket.joshi/.local/lib/python3.5/site-packages/ipykernel_launcher.py:4: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(8, (5, 5), kernel_initializer="he_uniform")`
  after removing the cwd from sys.path.


In [6]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 24, 24, 8)         208       
_________________________________________________________________
activation_1 (Activation)    (None, 24, 24, 8)         0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 20, 20, 8)         1608      
_________________________________________________________________
activation_2 (Activation)    (None, 20, 20, 8)         0         
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 10, 10, 8)         0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 800)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 128)               102528    
__________

In [19]:
model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer='adam',
              metrics=['accuracy'])


In [20]:
mcp_save = ModelCheckpoint('keras_model.hdf5', save_best_only=True, monitor='train_loss', mode='min', period = 5)
# model.fit_generator(train_generator(),steps_per_epoch = 100,validation_data = val_generator(), validation_steps = 50, epochs = 200,callbacks=[mcp_save])

In [21]:
model.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=200,
          verbose=1, callbacks=[mcp_save])

Epoch 1/200
60000/60000 [==============================] - 13s 210us/step - loss: 0.1681 - acc: 0.9500
Epoch 2/200
60000/60000 [==============================] - 3s 45us/step - loss: 0.0620 - acc: 0.9814
Epoch 3/200
60000/60000 [==============================] - 3s 45us/step - loss: 0.0487 - acc: 0.9849
Epoch 4/200
60000/60000 [==============================] - 3s 45us/step - loss: 0.0408 - acc: 0.9869
Epoch 5/200
60000/60000 [==============================] - 3s 44us/step - loss: 0.0355 - acc: 0.9885
Epoch 6/200
 3520/60000 [>.............................] - ETA: 2s - loss: 0.0322 - acc: 0.9881

/home/aniket.joshi/.local/lib/python3.5/site-packages/keras/callbacks.py:434: RuntimeWarning: Can save best model only with train_loss available, skipping.
  'skipping.' % (self.monitor), RuntimeWarning)


60000/60000 [==============================] - 3s 45us/step - loss: 0.0306 - acc: 0.9899
Epoch 7/200
60000/60000 [==============================] - 3s 44us/step - loss: 0.0268 - acc: 0.9910
Epoch 8/200
60000/60000 [==============================] - 3s 45us/step - loss: 0.0244 - acc: 0.9922
Epoch 9/200
60000/60000 [==============================] - 3s 44us/step - loss: 0.0205 - acc: 0.9934
Epoch 10/200
60000/60000 [==============================] - 3s 44us/step - loss: 0.0194 - acc: 0.9934
Epoch 11/200
60000/60000 [==============================] - 3s 45us/step - loss: 0.0179 - acc: 0.9944
Epoch 12/200
60000/60000 [==============================] - 3s 44us/step - loss: 0.0160 - acc: 0.9945
Epoch 13/200
60000/60000 [==============================] - 3s 45us/step - loss: 0.0159 - acc: 0.9947
Epoch 14/200
60000/60000 [==============================] - 3s 44us/step - loss: 0.0141 - acc: 0.9952
Epoch 15/200
60000/60000 [==============================] - 3s 45us/step - loss: 0.0134 - acc: 0.9

60000/60000 [==============================] - 3s 44us/step - loss: 0.0054 - acc: 0.9987
Epoch 87/200
60000/60000 [==============================] - 3s 45us/step - loss: 0.0065 - acc: 0.9982
Epoch 88/200
60000/60000 [==============================] - 3s 44us/step - loss: 0.0036 - acc: 0.9991
Epoch 89/200
60000/60000 [==============================] - 3s 45us/step - loss: 0.0076 - acc: 0.9982
Epoch 90/200
60000/60000 [==============================] - 3s 45us/step - loss: 0.0032 - acc: 0.9990
Epoch 91/200
60000/60000 [==============================] - 3s 44us/step - loss: 0.0049 - acc: 0.9989
Epoch 92/200
60000/60000 [==============================] - 3s 46us/step - loss: 0.0084 - acc: 0.9984
Epoch 93/200
60000/60000 [==============================] - 3s 46us/step - loss: 0.0040 - acc: 0.9988
Epoch 94/200
60000/60000 [==============================] - 3s 44us/step - loss: 0.0058 - acc: 0.9988
Epoch 95/200
60000/60000 [==============================] - 3s 45us/step - loss: 0.0051 - acc: 

60000/60000 [==============================] - 3s 43us/step - loss: 0.0062 - acc: 0.9990
Epoch 166/200
60000/60000 [==============================] - 3s 44us/step - loss: 0.0049 - acc: 0.9990
Epoch 167/200
60000/60000 [==============================] - 3s 43us/step - loss: 0.0061 - acc: 0.9991
Epoch 168/200
60000/60000 [==============================] - 3s 48us/step - loss: 0.0070 - acc: 0.9990
Epoch 169/200
60000/60000 [==============================] - 3s 47us/step - loss: 0.0054 - acc: 0.9992
Epoch 170/200
60000/60000 [==============================] - 3s 47us/step - loss: 0.0086 - acc: 0.9988
Epoch 171/200
60000/60000 [==============================] - 3s 46us/step - loss: 0.0074 - acc: 0.9989
Epoch 172/200
60000/60000 [==============================] - 3s 46us/step - loss: 0.0032 - acc: 0.9994
Epoch 173/200
60000/60000 [==============================] - 3s 47us/step - loss: 0.0055 - acc: 0.9992
Epoch 174/200
60000/60000 [==============================] - 3s 46us/step - loss: 0.010

In [22]:
score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])


Test loss: 0.15526142644580698
Test accuracy: 0.9871
